## Testing the VGG Face2 Algorithm on A2
The following document carries out a leave one out cross validation on Andreas Karges Album 2.

Install VGGFace2 by executing:

- pip install git+https://github.com/JohannesZahn/keras-vggface.git 
- pip install tensorflow==1.14.0
- pip install keras = 2.2.4

In [1]:
import pandas as pd
import os
from keras_vggface.vggface import VGGFace
from scipy.spatial.distance import cosine
from sklearn.metrics import accuracy_score
from IPython.display import Image
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from src import Config
import cv2
import keras_vggface
import tensorflow as tf

Using TensorFlow backend.
C:\Users\Johan\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Johan\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Johan\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Johan\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:


model = VGGFace(model='resnet50', include_top=False, input_shape=(224,224,3))

In [3]:
labels_df = pd.read_csv(Config.ANDREAS_ALBUMS_PATH / "labels.csv")
imageNames = os.listdir(Config.EXTRACTED_FACES_PATH)
imageCount = len(imageNames)
faces= np.zeros((imageCount, 224,224,3), dtype=np.float32)
for idx, image_name in enumerate(imageNames):
    image= cv2.imread(str(Config.EXTRACTED_FACES_PATH/image_name))
    resizedImage = cv2.resize(image, dsize=(224,224), interpolation=cv2.INTER_CUBIC)
    faces[idx, :,:,:] = resizedImage

In [5]:
tf.convert_to_tensor(faces,dtype=tf.float32)
embeddings = model.predict(faces)

01_0_0.png 01_0_0.png


#### Reappearing labels
* Persons can only be recognised in the test set, if they are seen at least once in the training dataset.  
    * Thus, the total dataset must have these people on two different photos.  
* Reappearing labels are candidates for the test data point during leave one out CV

In [13]:
def get_reappearing_labels(labels_df):
    return [label for label in labels_df.label.unique() if labels_df.label.value_counts()[label] >= 2]

Doing Leave-One-Out Cross Validation

In [15]:
reappearing_labels = get_reappearing_labels(labels_df)
predictions = []
ground_truth = []
for idx1, embedding1 in enumerate(embeddings):
    for idx2, embedding2 in enumerate(embeddings):
        if idx1!=idx2 and cosine(embedding1,embedding1)<0.3:
            label = labels_df.loc[labels_df['filename'] == imageNames[idx1]]['label']
            
            
            
    
for i, row in enumerate(labels_df.values):
    if row[1] in reappearing_labels:
        # Do Leave one out Cross Validation
        id, conf = predict_label(row[0], labels_df)
        predictions.append(id)
        ground_truth.append(row[1])

FileNotFoundError: [Errno 2] No such file or directory: 'dat\\AndreasAlbums\\extracted_faces\\01_1_0.png'

In [ ]:
acc = accuracy_score(predictions, ground_truth)
acc

Correcting the accuracy for duplicates (explanation below).

In [ ]:
N = len(predictions)
N_duplicates = 22
((acc * N)-N_duplicates)/(N-N_duplicates)

## Classification Mistakes
* Clearly the images have a bad resolution
    * I think this is the major bottleneck
* Some people are not directly facing the camera
* Few images are not faces

In [ ]:
plt.rcParams['xtick.labelsize'] = 0
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['ytick.labelsize'] = 0

**This image** was recognised as similar to **These images** but should have been classified as **Those images**.   

Image | Prediction class | Target class

In [ ]:
def row_in_plot(row, ax, prediction_label):
    for i in range(5):
        ax[i].axis('off')
        
    img = cv2.imread(os.path.join(path, row[0]))
    ax[0].title.set_text(row[0])
    ax[0].imshow(img)

    # Prediction class representation
    subset_df = labels_df[(labels_df.label == prediction_label) & (labels_df.filename != row[0])]
    prediction_class_filenames = subset_df.filename.tolist()
    img = cv2.imread(os.path.join(path, prediction_class_filenames[0]))
    ax[1].title.set_text(prediction_class_filenames[0])
    ax[1].imshow(img)

    img = cv2.imread(os.path.join(path, prediction_class_filenames[0]))
    ax[2].title.set_text(prediction_class_filenames[-1])
    ax[2].imshow(img)
    
    # Prediction class representation
    subset_df = labels_df[(labels_df.label == row[1]) & (labels_df.filename != row[0])]
    prediction_class_filenames = subset_df.filename.tolist()
    img = cv2.imread(os.path.join(path, prediction_class_filenames[0]))
    ax[3].title.set_text(prediction_class_filenames[0])
    ax[3].imshow(img)

    img = cv2.imread(os.path.join(path, prediction_class_filenames[0]))
    ax[4].title.set_text(prediction_class_filenames[-1])
    ax[4].imshow(img)

In [ ]:
cols = 5
# Never let rows < 2, as then pyplot decides to have a 1D list of axes, rather than 2D. (>.<)
rows = len([1 for i in range(len(predictions)) if predictions[i] != ground_truth[i]])

fig, ax = plt.subplots(rows, cols, figsize = (20,rows * 10))
plt.subplots_adjust(bottom=0.2, top=0.6, hspace=-0.3)
fig.suptitle('        Test img          |                            Prediction class                    |                           Target class               ', 
             fontsize=20, y = 0.6)

path = Config.EXTRACTED_FACES_PATH
predictions_idx = 0
row_idx = 0

for i, row in enumerate(labels_df.values):
    # if it reappears, then there is a prediction for it
    if row[1] in reappearing_labels:
        # if the prediction is wrong
        if predictions[predictions_idx] != ground_truth[predictions_idx] and row_idx < rows:
            row_in_plot(row, ax[row_idx], predictions[predictions_idx])
            row_idx += 1
        predictions_idx += 1
plt.savefig('LBPH mistakes on A2.png')

## Classification successes
* Duplication of pictures led to a higher accuracy
    * 16/28 were duplicates
* The silver lining is that face recognition works great when it's the same image
    * Hence it does work, just not that well
* Not sure how to automatically remove duplicates...
    * Could just do that by hand
    * But any manual process in the pipeline will be hard to automate later
    * Let's just say for now, that the lbg_hog algorithm wasnt very successful

In [ ]:
def row_in_plot2(row, ax, prediction_label):
    for i in range(5):
        ax[i].axis('off')
        
    img = cv2.imread(os.path.join(path, row[0]))
    ax[0].title.set_text(row[0])
    ax[0].imshow(img)

    # Target class representation
    subset_df = labels_df[(labels_df.label == row[1]) & (labels_df.filename != row[0])]
    prediction_class_filenames = subset_df.filename.tolist()
    img = cv2.imread(os.path.join(path, prediction_class_filenames[0]))
    ax[1].title.set_text(prediction_class_filenames[0])
    ax[1].imshow(img)

    img = cv2.imread(os.path.join(path, prediction_class_filenames[0]))
    ax[2].title.set_text(prediction_class_filenames[-1])
    ax[2].imshow(img)
    

In [ ]:

cols = 5
# Never let rows < 2, as then pyplot decides to have a 1D list of axes, rather than 2D. (>.<)
rows = len([1 for i in range(len(predictions)) if predictions[i] == ground_truth[i]])

fig, ax = plt.subplots(rows, cols, figsize = (20,rows * 10))
plt.subplots_adjust(bottom=0.2, top=0.6, hspace=-0.3)
fig.suptitle('Test img        |                           Target class                                                                                     ', 
             fontsize=20, y = 0.6)

path = Config.EXTRACTED_FACES_PATH
predictions_idx = 0
row_idx = 0

for i, row in enumerate(labels_df.values):
    # if it reappears, then there is a prediction for it
    if row[1] in reappearing_labels:
        # if the prediction is wrong
        if predictions[predictions_idx] == ground_truth[predictions_idx] and row_idx < rows:
            row_in_plot2(row, ax[row_idx], predictions[predictions_idx])
            row_idx += 1
        predictions_idx += 1

## Extraction Failures

As you can see from the success cases, occasionally the same face is extracted twice. The following section will outline some of those failures of the face extraction system.

In [ ]:
def plot_two_pics(filename1, filename2, path = Config.EXTRACTED_PHOTOS_PATH):
    fig, ax = plt.subplots(1, 2, figsize = (8,20))
    img = cv2.imread(os.path.join(path, filename1))
    ax[0].imshow(img)
    img = cv2.imread(os.path.join(path, filename2))
    ax[1].imshow(img)

### Failure: Photo extraction contains multiple photos
Occasionally, the photo extractor crops a too large section of the album page. This leads to inclusion of multiple photos in the extracted 'single photo'.  
Note, that both of the images of each pair displayed below was extracted as a "photo". The ones on the right are incorrectly extracted.

In [ ]:
plot_two_pics('45_2.png', '45_3.png')

In [ ]:
plot_two_pics('08_1.png', '08_2.png')

In [ ]:
plot_two_pics('06_0.png', '06_1.png')

In [ ]:
plot_two_pics('02_1.png', '02_2.png')

In [ ]:
plot_two_pics('08_1.png', '08_2.png')